In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from dynamicslib.common import prop, coupled_stm_eom,eom
from dynamicslib.integrate import dop853
from dynamicslib.consts import muEM
from dynamicslib.plotting import plotly_display, hodographs, to_eci, family_slider, animation_slider
import dynamicslib.common_targetters as targetters
import pandas as pd
from tqdm import tqdm
import os

# Set up CSV

In [ ]:
root = os.getcwd() + "/"
family = "Horseshoe A Axial 6"
params = [
    "Initial x",
    "Initial y",
    "Initial z",
    "Initial vx",
    "Initial vy",
    "Initial vz",
    "Period"
]
csv_data = pd.read_csv(root + family + ".csv")[:]
csv_data = csv_data[csv_data["Index"] >= 0]
for param in params:
    if param not in csv_data.columns:
        csv_data[param] = 0.0
csv_indexed = csv_data.set_index("Index")

# Propagate (for plotting)

In [ ]:
xyzs = []
vals = csv_indexed[params].values


for row in tqdm(vals):
    x0 = row[:-1]
    tf = row[-1]/2
    xyzs.append(prop(x0, tf, muEM, 1e-13, None))

# Plot the whole family

In [ ]:
# Requires prepropagation
a = None  # start
b = None  # end
c = 10  # step
plotly_display(xyzs[a:b:c], csv_data[a:b:c], port=8021)

# Plot one member at a time with animation

In [ ]:
# Does not require prepropagation
# Pick the appropriate targetter from targetters. Requires some knowledge of the family
# targetter = targetters.spatial_perpendicular(1e-13, muEM) # for on-the-fly targetting
targetter = targetters.fullstate_minus_one(2,0,0.,1e-13, muEM) # for on-the-fly targetting
# targetter = targetters.multi_shooter(15, 1e-13, muEM)

targ_tol = 1e-7 # targetting tolerance
animation_slider(csv_data, targetter, targ_tol, 1e-11, muEM, framerate=10)

# Plot in ECI

In [ ]:
# %matplotlib ipympl
# ts, xs, _, _ = dop853(eom, (0.0, tf*2), x0, 1e-16)
# out = to_eci(xs.T, ts)
# thta = np.linspace(0, 2 * np.pi, 1000)
# f = plt.figure()
# ax = f.add_subplot(projection="3d")
# ax.plot(np.cos(thta) * 384400, np.sin(thta) * 384400, 0 * thta, color="grey")
# ax.plot(out[:, 0], out[:, 1], out[:, 2], lw=1.5)
# ax.set(xlabel="ECI x [km]", ylabel="ECI y [km]", zlabel="ECI z [km]")
# ax.axis("equal")

# WIP/TODO

In [ ]:
# import pyvista as pv
# import vtk
# from pyvista.core.utilities import lines_from_points
# from pyvista.examples import mapfile

# pv.set_jupyter_backend("html")
# plotter = pv.Plotter()
# plotter.background_color = "white"
# c = pv.colors.get_cmap_safe("rainbow")
# # multi_block = pv.MultiBlock(xyzs)

# mbdata = {}
# for idx in range(len(xyzs))[::2]:
#     cx, cy, cz = xyzs[idx]
#     pts = np.column_stack((cx, cy, cz)).astype(np.float32)
#     line = lines_from_points(pts)
#     mbdata[f"line{idx}"] = line

# mb = pv.MultiBlock(mbdata)
# plotter.add_mesh(mb)
# earth = pv.Sphere(radius=6371 / 384400, center=[-muEM, 0, 0])
# earth.texture_map_to_sphere(inplace=True)
# image_file = mapfile
# tex = pv.read_texture(image_file)
# plotter.add_mesh(earth, texture=tex)

# moon = pv.Sphere(radius=1740 / 384400, center=[1 - muEM, 0, 0])
# moon.texture_map_to_sphere(inplace=True)
# image_file = mapfile
# tex = pv.read_texture("moon.jpg")
# plotter.add_mesh(moon, texture=tex)

# plotter.show_bounds(xtitle="$x$ [LU]", ytitle="$y$ [LU]", ztitle="$z$ [LU]")
# # plotter.show_bounds()
# plotter.show()
# # plotter.show(jupyter_backend="html")